In [4]:
from pydub import AudioSegment 

song = AudioSegment.from_wav("og.wav") 

# pydub does things in milliseconds 
ten_seconds = 10 * 3

first_10_seconds = song[:ten_seconds] 

last_5_seconds = song[-5000:] 

first_10_seconds.export("new.wav", format="wav")

AudioSegment.from_wav("og.wav").export("file.mp3", format="mp3")

FileNotFoundError: [WinError 2] The system cannot find the file specified